# 베이스라인 모델

In [ ]:
import pandas as pd

In [ ]:
# 데이터 경로
data_path = '/kaggle/input/porto-seguro-safe-driver-prediction/'

train = pd.read_csv(data_path + 'train.csv', index_col='id')
test = pd.read_csv(data_path + 'test.csv', index_col='id')
submission = pd.read_csv(data_path + 'sample_submission.csv', index_col='id')

## 피처 엔지니어링

### 데이터 합치기

In [ ]:
all_data = pd.concat([train, test], ignore_index=True)
all_data = all_data.drop('target', axis=1)  # 타깃값 제거

In [ ]:
all_features = all_data.columns  # 전체 피처
all_features

### 명목형 피처 원-핫 인코딩

모든 명목형 피처에 원-핫 인코딩을 적용

명목형 데이터에는 고윳값별 순서가 따로 없음

'cat' 이 포함된 피처가 명목형 피처임

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
# 명목형 피처 추출
cat_features = [feature for feature in all_features if 'cat' in feature]

onehot_encoder = OneHotEncoder()  # 원-핫 인코더 객체 생성

# 인코딩
encoded_cat_matrix = onehot_encoder.fit_transform(all_data[cat_features])

encoded_cat_matrix

### 필요 없는 피처 제거

- 이진 피처 : ps_ind_10_bin ~ ps_ind_13_bin, ps_calc_15_bin ~ ps_calc_20_bin
- 순서형 피처 : ps_ind_14, ps_calc_04 ~ ps_calc_14
- 연속형 피처 : ps_calc_01 ~ ps_calc_03, ps_car_14

In [ ]:
# 추가로 제거할 피처
drop_features = ['ps_ind_14', 'ps_ind_10_bin', 'ps_ind_11_bin', 'ps_ind_12_bin', 'ps_ind_13_bin', 'ps_car_14']

# '1) 명목형 피처, 2) calc 분류의 피처, 3) 추가 제거할 피처'를 제외한 피처
remaining_features = [feature for feature in all_features if ('cat' not in feature and 'calc' not in feature and feature not in drop_features)]

remaining_features

인코딩과 피처 제거가 끝났음

In [ ]:
from scipy import sparse

In [ ]:
all_data_sprs = sparse.hstack([sparse.csr_matrix(all_data[remaining_features]),
                               encoded_cat_matrix],
                              format='csr')

> csr_matrix() 는 전달받은 데이터를 CSR 형식으로 바꿔줍니다. hstack() 은 행렬을 수평 방향으로 합칩니다.

### 데이터 나누기

In [ ]:
num_train = len(train)  # 훈련 데이터 개수

# 훈련 데이터와 테스트 데이터 나누기
X = all_data_sprs[:num_train]
X_test = all_data_sprs[num_train:]

y = train['target'].values

## 평가지표 계산 함수 작성

### 지니계수란?

원래 경제학에서 쓰는 용어로 소득 불평등 정도를 나타내는 지표임

지니계수가 작을수록 소득 수준이 평등하고, 클수록 불평등함을 의미

지니계수는 로렌츠 곡선을 이용해 계산하고 로렌츠 곡선을 그리려면 모든 경제인구를 소득 순서대로 나열한 후에 가로축은 인구 누적 비율, 세로축은 소득 누적 점유율로 설정

인구 누적 비율과 해당 소득 누적 점유율을 연결한 선을 로렌츠 곡선이라고 함

<img src=https://t1.daumcdn.net/cfile/blog/224431355845363417>

지니계수는 A 영역 넓이를 삼각형 전체 넓이로 나눈 값을 의미

A 영역이 좁을수록(로렌츠 곡선이 대각선과 가까워질수록) 소득 수준은 평등

반대로 A 영역이 넓을수록(로렌츠 곡선이 대각선과 멀어질수록) 소득 수준은 불평등

머신러닝에서 지니계수는 모델의 예측 성능을 측정하는데 쓰임

예측값을 크기순으로 정렬해서 로렌츠 곡선을 구함

> 지니계수 값은 (2 x ROC AUC - 1)과 같음. 그렇기 때문에 평가지표가 지니계수이면 평가지표가 ROC AUC 인 상황과 거의 비슷하긴 함

### 정규화 지니계수 계산 함수

정규화란 값의 범위를 0~1 사이로 조정한다는 뜻이므로, 정규화 지니계수는 값이 0에 가까울수록 성능이 나쁘고, 1에 가까울수록 성능이 좋다는 의미

$$정규화 지니계수 = \frac{예측 값에 대한 지니계수}{예측이 완벽할 떄의 지니계수}$$

'예측 값에 대한 지니계수' 는 예측값과 실제값으로 구한 지니계수

'예측이 완벽할 때의 지니계수' 는 실젯값과 실젯값으로 구한 지니계수를 뜻함

In [ ]:
import numpy as np

In [ ]:
def eval_gini(y_true, y_pred):
    # 실젯값과 예측값의 크기가 서로 같은지 확인 (값이 다르면 오류 발생)
    assert y_true.shape == y_pred.shape
    
    n_samples = y_true.shape[0]  # 데이터 개수
    L_mid = np.linspace(1 / n_samples, 1, n_samples)  # 대각선 값
    
    # 1) 예측값에 대한 지니계수
    pred_order = y_true[y_pred.argsort()]  # y_pred 크기순으로 y_true 값 정렬
    L_pred = np.cumsum(pred_order) / np.sum(pred_order)  # 로렌츠 곡선
    G_pred = np.sum(L_mid - L_pred)  # 예측값에 대한 지니계수
    
    # 2) 예측이 완벽할 때 지니계수
    true_order = y_true[y_true.argsort()]  # y_true 크기순으로 y_true 값 정렬
    L_true = np.cumsum(true_order) / np.sum(true_order)  # 로렌츠 곡선
    G_true = np.sum(L_mid - L_true)  # 예측이 완벽할 때 지니계수
    
    # 정규화된 지니계수
    return G_pred / G_true

> 로직을 간단히만 설명해보자면 로렌츠 곡선 상단 넓이를 삼각형 넓이로 나눈 값이 지니계수임. 정규화 지니계수는 '예측값에 대한 지니계수'를 '예측이 완벽할 때의 지니계수'로 나눈 값임. 결국 로렌츠 곡선 상단 넓이를 구할 수 있으면 정규화 지니계수도 구할 수 있음

In [ ]:
# LightGBM 용 gini() 함수
def gini(preds, dtrain):
    labels = dtrain.get_label()
    return 'gini', eval_gini(labels, preds), True  # 반환값(평가지표 이름, 평가 점수, 평가 점수가 높을수록 좋은지 여부)

## 모델 훈련 및 성능 검증

### OOF 예측 방식

OOF 예측(Out of Fold prediction)이란 K 폴드 교차 검증을 수행하면서 각 폴드마다 테스트 데이터로 예측하는 방식

K 폴드 교차 검증을 하면서 폴드마다 1) 훈련 데이터로 모델을 훈련하고 -> 2) 검증 데이터로 모델 성능을 측정하며 -> 3) 테스트 데이터로 최종 타깃 확률도 예측

훈련된 모델로 마지막에 한 번만 예측하는게 아니고 각 폴드별 모델로 여러번 예측해 평균을 내는 방식

예측 절차는 다음과 같음

1. 전체 훈련 데이터를 K 개 그룹으로 나눔
2. K 개 그룹 중 한 그룹은 검증 데이터, 나머지 K-1 개 그룹은 훈련 데이터로 지정
3. 훈련 데이터로 모델을 훈련
4. 훈련된 모델을 이용해 검증 데이터로 타깃 확률을 예측하고, 전체 테스트 데이터로도 타깃 확률을 예측
5. 검증 데이터로 구한 예측 확률과 테스트 데이터로 구한 예측 확률을 기록
6. 검증 데이터를 다른 그룹으로 바꿔가며 2~5번 절차를 총 K 번 반복
7. K 개 그룹의 검증 데이터로 예측한 확률을 훈련 데이터 실제 타깃값과 비교해 성능 평가점수를 계산. 이 점수로 모델 성능을 가늠해볼 수 있음
8. 테스트 데이터로 구한 K 개 예측 확률의 평균을 구함. 이 값이 최종 예측 확률이며, 제출해야 하는 값임

두 가지 장점

1. 과대적합 방지 효과
2. 앙상블 효과

### OOF 방식으로 LightGBM 훈련

In [ ]:
from sklearn.model_selection import StratifiedKFold

In [ ]:
# 층화 K 폴드 교차 검증기
folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=1991)

> 시계열 데이터는 순서가 중요해서 데이터를 섞으면 안됨. 데이터를 섞으면 순서에 담긴 의미가 사라짐

In [ ]:
params = {'objective': 'binary',
          'learning_rate': 0.01,
          'force_row_wise': True,  # 경고문구 제거
          'random_state': 0}

In [ ]:
# OOF 방식으로 훈련된 모델로 검증 데이터 타깃값을 예측한 화귤ㄹ을 담을 1차원 배열
oof_val_preds = np.zeros(X.shape[0])
# OOF 방식으로 훈련된 모델로 테스트 데이터 타깃값을 예측한 확률을 담을 1차원 배열
oof_test_preds = np.zeros(X_test.shape[0])

In [ ]:
import lightgbm as lgb

In [ ]:
# OOF 방식으로 모델 훈련, 검증, 예측
for idx, (train_idx, valid_idx) in enumerate(folds.split(X, y)):
    # 각 폴드를 구분하는 문구 출력
    print('#' * 40, f'폴드 {idx+1} / 폴드 {folds.n_splits}', '#' * 40)
    
    # 훈련용 데이터, 검증용 데이터 설정
    X_train, y_train = X[train_idx], y[train_idx]  # 훈련용 데이터
    X_valid, y_valid = X[valid_idx], y[valid_idx]  # 검증용 데이터
    
    # LightGBM 전용 데이터셋 생성
    dtrain = lgb.Dataset(X_train, y_train)  # LightGBM 전용 훈련 데이터셋
    dvalid = lgb.Dataset(X_valid, y_valid)  # LightGBM 전용 검증 데이터셋
    
    # LightGBM 모델 훈련
    lgb_model = lgb.train(params=params,  # 훈련용 하이퍼파라미터
                          train_set=dtrain,  # 훈련 데이터셋
                          num_boost_round=1000,  # 부스팅 반복 횟수
                          valid_sets=dvalid,  # 성능 평가용 검증 데이터셋
                          feval=gini,  # 검증용 평가지표
                          early_stopping_rounds=100,  # 조기종료 조건
                          verbose_eval=100,  # 100번쨰마다 점수 출력
                         )  
    
    # 테스트 데이터를 활용해 OOF 예측
    oof_test_preds += lgb_model.predict(X_test) / folds.n_splits
    # 모델 성능 평가를 위한 검증 데이터 타깃값 예측
    oof_val_preds[valid_idx] += lgb_model.predict(X_valid)
    
    # 검증 데이터 예측 확률에 대한 정규화 지니계수
    gini_score = eval_gini(y_valid, oof_val_preds[valid_idx])
    print(f'폴드 {idx+1} 지니계수: {gini_score}\n')

> 파이썬 래퍼 LightGBM 이 제공하는 predict() 메서드에 Dataset 타입을 젇날하면 오류가 발생

> feval 파라미터에 사용자 정의 함수를 전달하면 이 함수의 값을 평가지표로 삼아 조기종료가 적용

In [ ]:
print('OOF 검증 데이터 지니계수:', eval_gini(y, oof_val_preds))

## 예측 및 결과 제출

In [ ]:
submission['target'] = oof_test_preds
submission.to_csv('submission.csv')